# BLSTM Notebook

In [9]:
import pandas as pd
import numpy as np
from model_trainer_and_tester import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, Input, TimeDistributed
from model.scoring_metrics import get_windiff, get_pk, get_k_kappa
from model.load_data import train_test_split_LSTM
from model_trainer_and_tester import read_in_dataset_lstm

In [10]:
batch_size = 64
n_timesteps = 20
train_ratio = 0.4 # ratio of meetings the model is trained on
LSTM_units = 20

features = ['pause','speakerChange', 'similarity', 'f0_diff', 'f0_baseline_diff']
shift = [-1, 0, 1]

datasets = """Bed002 Bed003 Bed004 Bed005 Bed006 Bed008 Bed009 Bed010 Bed011 Bed012 Bed013 Bed014 Bed015 Bed016 Bed017 Bmr001 Bmr002 Bmr005 Bmr007 Bmr009 Bmr010 Bmr011 Bmr012 Bmr013 Bmr014 Bmr018 Bmr019 Bmr021 Bmr022 Bmr024 Bmr025 Bmr026 Bmr027 Bmr029 Bns001 Bns002""".split(" ")
results_merged_path = "../results_merged_fixedf0/"

In [4]:
#with overlap

X_train, Y_train = read_in_dataset_lstm(features, shift, to_read='train')
X_test, Y_test = read_in_dataset_lstm(features, shift, to_read='test')

X_train = X_train.astype(float)
X_test = X_test.astype(float)

n_timesteps = len(shift)

c:\Users\janwe\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
c:\Users\janwe\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [11]:
#without overlap


#load the data
X_train_df, Y_train_df, X_test_df, Y_test_df= train_test_split_LSTM(datasets, results_merged_path, n_timesteps, split=train_ratio)

# convert data from dataframes into numpy arrays of in a 3-D shape (samples, timesteps, features)
X_train = X_train_df.values.astype('float32').reshape(-1, n_timesteps, 5)
X_test = X_test_df.values.astype('float32').reshape(-1, n_timesteps, 5)
Y_train = Y_train_df.values.astype('float32').reshape(-1, n_timesteps, 1)
Y_test = Y_test_df.values.astype('float32').reshape(-1, n_timesteps, 1)

In [12]:
model = Sequential()
model.add(Bidirectional(LSTM(LSTM_units, return_sequences=True), input_shape=(n_timesteps, len(features))))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the model
history=model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=40,
                  validation_data=[X_test, Y_test])

#measure the performance
predictions = model(X_test).numpy()

Epoch 1/40
16/16 [==============================] - 4s 60ms/step - loss: 9.9240e-10 - accuracy: 0.0692 - val_loss: 1.0450e-09 - val_accuracy: 0.0901
Epoch 2/40
16/16 [==============================] - 0s 17ms/step - loss: 9.9240e-10 - accuracy: 0.0692 - val_loss: 1.0450e-09 - val_accuracy: 0.0901
Epoch 3/40
16/16 [==============================] - 0s 16ms/step - loss: 9.9240e-10 - accuracy: 0.0692 - val_loss: 1.0450e-09 - val_accuracy: 0.0901
Epoch 4/40
16/16 [==============================] - 0s 18ms/step - loss: 9.9240e-10 - accuracy: 0.0692 - val_loss: 1.0450e-09 - val_accuracy: 0.0901
Epoch 5/40
16/16 [==============================] - 0s 14ms/step - loss: 9.9240e-10 - accuracy: 0.0692 - val_loss: 1.0450e-09 - val_accuracy: 0.0901
Epoch 6/40
16/16 [==============================] - 0s 17ms/step - loss: 9.9240e-10 - accuracy: 0.0692 - val_loss: 1.0450e-09 - val_accuracy: 0.0901
Epoch 7/40
16/16 [==============================] - 0s 17ms/step - loss: 9.9240e-10 - accuracy: 0.0692 - v

In [26]:
Y_test_eval = Y_test[:, 1, 0]
pred_eval = predictions[:, 1, 0]
# I'll just be lazy for pred_eval, if bigger or equal to 0.5 than it's 1, else it's 0
pred_eval = (pred_eval >= 0.1).astype(float)

In [21]:
Y_test_eval = Y_test.flatten()
pred_eval = predictions.flatten()
pred_eval = (pred_eval >= 0.5).astype(float)

In [23]:
print(predictions)
print(pred_eval)

[[[0.5506092 ]
  [0.5588558 ]
  [0.5474897 ]
  ...
  [0.54922056]
  [0.5511437 ]
  [0.53763413]]

 [[0.57717365]
  [0.5761189 ]
  [0.56234396]
  ...
  [0.5559836 ]
  [0.5248605 ]
  [0.5384692 ]]

 [[0.5058322 ]
  [0.49426886]
  [0.505665  ]
  ...
  [0.56055427]
  [0.54725194]
  [0.5481628 ]]

 ...

 [[0.5732085 ]
  [0.5650766 ]
  [0.56385624]
  ...
  [0.4329162 ]
  [0.5141409 ]
  [0.49796537]]

 [[0.560122  ]
  [0.5783169 ]
  [0.58428764]
  ...
  [0.5599628 ]
  [0.54653054]
  [0.5329223 ]]

 [[0.5452558 ]
  [0.53671676]
  [0.5366806 ]
  ...
  [0.5188657 ]
  [0.50154805]
  [0.46091148]]]
[1. 1. 1. ... 1. 1. 0.]


In [19]:
print(Y_test.shape)
print(pred_eval.shape)
print(Y_test_eval.shape)
print(predictions.shape)
print(int(round(len(pred_eval) / (np.count_nonzero(pred_eval) * 2.0))))

(1540, 20, 1)
(30800,)
(30800,)
(1540, 20, 1)
0


In [15]:
windiff = get_windiff(Y_test_eval, pred_eval)
pk = get_pk(Y_test_eval, pred_eval, k=58)
k_k = get_k_kappa(Y_test_eval, pred_eval)
print("windiff:", windiff)
print("pk:", pk)
print("K_k ", k_k)

windiff: 1.0
pk: 0.599030673649286
K_k  0.0
